In [1]:
!pip3 install editdistance==0.5.3
!pip3 install numba==0.49.0

     |████████████████████████████████| 179 kB 4.6 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 3.6 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 20.2 MB 20.6 MB/s eta 0:00:01     |███████████████████▍            | 12.2 MB 20.6 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import string
import datetime

from data.generator import DataGenerator, Tokenizer
from network.model import HTRModel

In [41]:
input_size = (128, 64, 1)
max_text_length = 32
charset_base = string.printable[:95]

source='iam'
arch ="flor" # puigcerver, bluche, flor,
batch_size = 100
type_of_run = 'train'
if type_of_run == 'train':
    train_model = False
else:
    train_model = True

In [42]:
source_path = os.path.join("/floyd/input/words_htr_onew", f"{source}_tvt.hdf5")
output_path = os.path.join("/floyd/home/output_words", source, arch)
target_path = os.path.join(output_path, "checkpoint_weights.hdf5")

assert os.path.isfile(source_path) or os.path.isfile(target_path)
os.makedirs(output_path, exist_ok=True)

In [43]:
dtgen = DataGenerator(source=source_path,
                      batch_size=batch_size,
                      charset=charset_base,
                      max_text_length=max_text_length,
                      predict=train_model)

model = HTRModel(architecture=arch,
                 input_size=input_size,
                 vocab_size=dtgen.tokenizer.vocab_size,
                 beam_width=10,
                 stop_tolerance=20,
                 reduce_tolerance=15)

model.compile(learning_rate=0.001)
model.load_checkpoint(target=target_path)

model.summary(output_path, "summary.txt")
callbacks = model.get_callbacks(logdir=output_path, checkpoint=target_path, verbose=1)

/floyd/input/words_htr_onew/iam_tvt.hdf5
Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 128, 64, 1)]      0         
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 64, 32, 16)        160       
_________________________________________________________________
p_re_lu_90 (PReLU)           (None, 64, 32, 16)        16        
_________________________________________________________________
batch_normalization_95 (Batc (None, 64, 32, 16)        112       
_________________________________________________________________
full_gated_conv2d_75 (FullGa (None, 64, 32, 16)        4640      
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 64, 32, 32)        4640      
_________________________________________________________________
p_re_lu_91 (PReLU

In [44]:
print(f"Train images: {dtgen.size['train']}")
print(f"Valid images: {dtgen.size['valid']}")
print(f"Test images: {dtgen.size['test']}")

Train images: 53782
Valid images: 5963
Test images: 6603


In [ ]:
start_time = datetime.datetime.now()

h = model.fit(x=dtgen.next_train_batch(),
              epochs=400,
              steps_per_epoch=dtgen.steps['train'],
              validation_data=dtgen.next_valid_batch(),
              validation_steps=dtgen.steps['valid'],
              callbacks=callbacks,
              shuffle=True,
              verbose=1)

total_time = datetime.datetime.now() - start_time

loss = h.history['loss']
val_loss = h.history['val_loss']

min_val_loss = min(val_loss)
min_val_loss_i = val_loss.index(min_val_loss)

time_epoch = (total_time / len(loss))
total_item = (dtgen.size['train'] + dtgen.size['valid'])

t_corpus = "\n".join([
    f"Total train images:      {dtgen.size['train']}",
    f"Total validation images: {dtgen.size['valid']}",
    f"Batch:                   {dtgen.batch_size}\n",
    f"Total time:              {total_time}",
    f"Time per epoch:          {time_epoch}",
    f"Time per item:           {time_epoch / total_item}\n",
    f"Total epochs:            {len(loss)}",
    f"Best epoch               {min_val_loss_i + 1}\n",
    f"Training loss:           {loss[min_val_loss_i]:.8f}",
    f"Validation loss:         {min_val_loss:.8f}"
])

with open(os.path.join(output_path, "train.txt"), "w") as lg:
    lg.write(t_corpus)
    print(t_corpus)

Epoch 1/400
538/538 [==============================] - ETA: 0s - loss: 15.2639
Epoch 00001: val_loss improved from inf to 8.97005, saving model to /floyd/home/output_words/iam/flor/checkpoint_weights.hdf5
538/538 [==============================] - 114s 212ms/step - loss: 15.2639 - val_loss: 8.9701 - lr: 0.0010
Epoch 2/400
538/538 [==============================] - ETA: 0s - loss: 15.2591
Epoch 00002: val_loss improved from 8.97005 to 8.96807, saving model to /floyd/home/output_words/iam/flor/checkpoint_weights.hdf5
538/538 [==============================] - 112s 208ms/step - loss: 15.2591 - val_loss: 8.9681 - lr: 0.0010
Epoch 3/400
538/538 [==============================] - ETA: 0s - loss: 15.2603
Epoch 00003: val_loss did not improve from 8.96807
538/538 [==============================] - 112s 208ms/step - loss: 15.2603 - val_loss: 9.0513 - lr: 0.0010
Epoch 4/400
538/538 [==============================] - ETA: 0s - loss: 15.2357
Epoch 00004: val_loss did not improve from 8.96807
538/5